## FF++ training data rawframes2

In [ ]:
import os
import cv2
import json

## need modify dataset saving location on your device and output training list name
base_dirs_manipulated = [
    '/home/jovyan/dataset/FaceForensics++/manipulated_sequences/youtube/c23/rawframes2_random_margin',
    '/home/jovyan/dataset/FaceForensics++/manipulated_sequences/Deepfakes/c23/rawframes2_random_margin',
    '/home/jovyan/dataset/FaceForensics++/manipulated_sequences/Face2Face/c23/rawframes2_random_margin',
    '/home/jovyan/dataset/FaceForensics++/manipulated_sequences/FaceSwap/c23/rawframes2_random_margin',
    '/home/jovyan/dataset/FaceForensics++/manipulated_sequences/NeuralTextures/c23/rawframes2_random_margin'
]
base_dir_original = '/home/jovyan/dataset/FaceForensics++/original_sequences/youtube/c23/rawframes2_random_margin'

output_file = 'rawframes2_random_margin_5frames.txt'
manipulated_line_counts = {}
original_line_counts = {}


with open(file=os.path.join(
        os.path.join("/home/jovyan/dataset/FaceForensics++/",
                     "train.json")),
          mode="r") as f:
    train_json = json.load(f)
train_list = []
for d1, d2 in train_json:
    train_list.append(d1)
    train_list.append(d2)
    train_list.append(d1 + "_" + d2)
    train_list.append(d2 + "_" + d1)

with open(output_file, 'w') as file:
    for base_dir in base_dirs_manipulated:
        total_lines = 0
        subdirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d)) and '.ipynb' not in d and d in train_list]
        for subdir in subdirs:
            subsubdirs = [os.path.join(subdir, d) for d in os.listdir(subdir) if os.path.isdir(os.path.join(subdir, d)) and '.ipynb' not in d]
            if len(subsubdirs) < 1:
                continue
            for subsubdir in subsubdirs:
                files = [p for p in os.listdir(subsubdir) if p.endswith('.png')]
                image_count = len(files)
                relative_path = subsubdir.replace('dataset/FaceForensics++/', '')

                if image_count >= 32:
                    start_frame = 0
                    end_frame = image_count
                    file.write(f"{subsubdir} {start_frame} {end_frame} 1\n")
                    total_lines += 1
        manipulated_line_counts[base_dir] = total_lines


    subdirs = [os.path.join(base_dir_original, d) for d in os.listdir(base_dir_original) if os.path.isdir(os.path.join(base_dir_original, d)) and '.ipynb' not in d and d in train_list]
    total_lines = 0
    for subdir in subdirs:
        subsubdirs = [os.path.join(subdir, d) for d in os.listdir(subdir) if os.path.isdir(os.path.join(subdir, d)) and '.ipynb' not in d]
        for subsubdir in subsubdirs:
            files = [p for p in os.listdir(subsubdir) if p.endswith('.png')]
            image_count = len(files)
            if image_count >= 32:
                start_frame = 0
                end_frame = image_count
                file.write(f"{subsubdir} {start_frame} {end_frame} 0\n")
                total_lines += 1
    original_line_counts[base_dir_original] = total_lines

    for base_dir, count in manipulated_line_counts.items():
        relative_base_dir = base_dir.replace('dataset/FaceForensics++', '')
        print(f"{relative_base_dir}: {count}")

    for base_dir, count in original_line_counts.items():
        relative_base_dir = base_dir.replace('dataset/FaceForensics++', '')
        print(f"{relative_base_dir}: {count}")

print(f"文件已生成：{output_file}")


## Celebdf V2

In [ ]:
import os

def read_celebdf_label_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(' ')
            label = 1 if parts[0] == '0' else 0
            original_path = parts[1]
            ## need modify dataset saving location on your device
            new_path = '/home/jovyan/dataset/Celeb-DF-v2/' + '/'.join(original_path.split('/')[:-1]) + '/rawframes2_test/' + original_path.split('/')[-1].replace('.mp4', '')

            data.append((int(label), new_path))
    return data


def list_dirs_and_write_new_file(data, new_file_path):
    total_clip = 0
    total_identity = 0
    with open(new_file_path, 'w') as file:
        for label, path in data:
            # 列出clip目錄
            subdir = [os.path.join(path, d) for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))and '.ipynb' not in d]
            if len(subdir)<=1:
                continue
            
            
            for subsubdir in subdir:
                # 列出identity目錄
                total_clip += 1
                face_dirs =  [os.path.join(subsubdir, d) for d in os.listdir(subsubdir) if os.path.isdir(os.path.join(subsubdir, d)) and '.ipynb' not in d]
                
                for face_dir in face_dirs:
                    total_identity += 1
                    files = [p for p in os.listdir(face_dir) if p.endswith('.png')]
                    files.sort()
                    image_count = len([f for f in files if f.endswith(('.png'))])

                    if image_count >= 16:
                        start_frame = 0
                        end_frame = image_count
                        file.write(f"{face_dir} {start_frame} {end_frame} {label}\n")


    print(f"total_clip: {total_clip},  total_identity: {total_identity}")

## need modify dataset saving location on your device and output training list name
original_file_path = '/home/jovyan/dataset/Celeb-DF-v2/List_of_testing_videos.txt'
new_file_path = 'rawframes2_cdf_identity.txt'
data = read_celebdf_label_file(original_file_path)
list_dirs_and_write_new_file(data, new_file_path)


## DFDC

In [ ]:
import os
import pandas as pd
import random

def list_direct_subfolders(path):
    # This function will list only the direct subfolders of the given path
    all_video = [
        os.path.join(path, d) for d in os.listdir(path)
        if os.path.isdir(os.path.join(path, d))
    ]
    all_video.sort()
    return all_video

def read_gt_csv(csv_path):
    df = pd.read_csv(csv_path)
    real_list = df.loc[df['label'] == 0, 'filename'].tolist()
    real_list = [video.split(".")[0] for video in real_list]
    fake_list = df.loc[df['label'] == 1, 'filename'].tolist()
    fake_list = [video.split(".")[0] for video in fake_list]

    return real_list, fake_list

## need modify dataset saving location on your device and output training list name
real_list, fake_list = read_gt_csv("/home/jovyan/dataset/DFDC/test/labels.csv")
base_path = "/home/jovyan/dataset/DFDC/test/rawframes2_test_2"
folders = list_direct_subfolders(base_path)
output_path = 'test_list_DFDC_full_with_identity.txt'

with open(output_path, 'w') as file:
    for folder in folders:
        video_name = os.path.basename(folder)
        subfolders = [os.path.join(folder , d) for d in os.listdir(folder) if '.ipynb' not in d]
        if len(subfolders) < 1:
            continue
        for subsubdir in subfolders:
            face_dirs =  [os.path.join(subsubdir, d) for d in os.listdir(subsubdir) if os.path.isdir(os.path.join(subsubdir, d)) and '.ipynb' not in d]
            for face_dir in face_dirs:
                files = [p for p in os.listdir(face_dir) if p.endswith('.png')]
                image_count = len(files)
                if image_count >= 16:
                    start_frame = 0
                    if video_name in real_list:
                        file.write(f'{face_dir} {start_frame} {image_count} 0\n')
                    elif video_name in fake_list:
                        file.write(f'{face_dir} {start_frame} {image_count} 1\n')


